# Denoiser

This notebook assumes you have training images in `/training`.

In [ ]:
import os
import numpy as np
import cv2
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence

In [ ]:
def load_images(folder, size=(256, 256)):
    images = []
    files = os.listdir(folder)
    random.shuffle(files)
    for file in files:
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, size)
        images.append(img)
    return np.array(images)


def generate_jpeg_images(images, quality=60):
    jpeg_images = []
    for img in images:
        _, encoded_img = cv2.imencode(".jpg", img, [cv2.IMWRITE_JPEG_QUALITY, quality])
        decoded_img = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR)
        jpeg_images.append(decoded_img)
    return np.array(jpeg_images)


folder = "/content/training"
source_images = load_images(folder)
jpeg_images = generate_jpeg_images(source_images)

X_train, X_test, y_train, y_test = train_test_split(
    jpeg_images, source_images, test_size=0.2, random_state=42
)


In [ ]:
# Normalize the data
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
y_train = y_train.astype("float32") / 255.0
y_test = y_test.astype("float32") / 255.0


In [ ]:
folder = "/images/train"
batch_size = 16
data_generator = ImageDataGenerator(folder, batch_size)

X_train = data_generator
y_train = None

X_test = []
y_test = []

test_files = os.listdir("/images/test")
for file in test_files:
    img_path = os.path.join("/images/test", file)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256))
    X_test.append(img / 255.0)

    _, encoded_img = cv2.imencode(
        ".jpg", img, [cv2.IMWRITE_JPEG_QUALITY, random.randint(20, 80)]
    )
    decoded_img = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR)
    y_test.append(decoded_img / 255.0)

X_test = np.array(X_test)
y_test = np.array(y_test)


In [ ]:
def unet_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    conv1 = Conv2D(64, 3, activation="relu", padding="same")(inputs)
    conv1 = Conv2D(64, 3, activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv_last = Conv2D(64, 3, activation="relu", padding="same")(pool1)
    conv_last = Conv2D(64, 3, activation="relu", padding="same")(conv_last)
    up_last = UpSampling2D(size=(2, 2))(conv_last)
    merge_last = concatenate([conv1, up_last], axis=3)
    conv_last = Conv2D(64, 3, activation="relu", padding="same")(merge_last)
    conv_last = Conv2D(64, 3, activation="relu", padding="same")(conv_last)

    conv_out = Conv2D(3, 1, activation="linear")(
        conv_last
    )

    return Model(inputs=inputs, outputs=conv_out)


model = unet_model()
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])


In [ ]:
checkpoint = ModelCheckpoint(
    "denoiser-{epoch:02d}.h5",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    mode="min",
)

history = model.fit(
    X_train,
    y_train,
    batch_size=16,
    epochs=20,
    verbose=1,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint],
)


In [ ]:
model.save("denoiser.h5")
